##데이터 임포트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Pycret/장애인취업정보.csv', encoding='euc-kr')

In [4]:
df['근무지역'] = df['근무지역'].str[:2]
df = df.dropna()

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
ml_df = df[['연령', '장애유형', '중증여부', '근무지역','취업직종대분류']].copy()

In [8]:
!pip install pycaret --quiet
!pip install shap --quiet
!pip install --upgrade pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.3 MB/s e

In [7]:
from sklearn.model_selection import train_test_split

# 데이터 제거 (둘다 2개씩 있어서 지움)
ml_df = ml_df[ml_df['취업직종대분류'] != '인문·사회과학 연구직']
ml_df = ml_df[ml_df['취업직종대분류'] != '자연·생명과학 연구직']

# 제거 후 데이터 확인
print(ml_df['취업직종대분류'].value_counts())

경영·행정·사무직                            16940
청소 및 기타 개인서비스직                       11692
제조 단순직                                3230
경호·경비직                                2162
보건·의료직                                2012
음식 서비스직                               1562
돌봄 서비스직(간병·육아)                         966
인쇄·목재·공예 및 기타 설치·정비·생산직                883
영업·판매직                                 680
스포츠·레크리에이션직                            625
전기·전자 설치·정비·생산직                        568
관리직(임원·부서장)                            448
예술·디자인·방송직                             436
식품 가공·생산직                              416
사회복지·종교직                               406
운전·운송직                                 361
기계 설치·정비·생산직                           349
화학·환경 설치·정비·생산직                        161
건설·채굴직                                 155
제조 연구개발직 및 공학기술직                       150
섬유·의복 생산직                              147
교육직                                    145
미용·예식 서비스직                             142
농림어업직      

In [9]:
!pip install --upgrade scipy
!pip install --upgrade pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pycaret 3.2.0 requires scipy~=1.10.1, but you have scipy 1.11.4 which is incompatible.
  Using cached scipy-1.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages 

## Pycaret 시작

In [11]:
# 이거 안돌아가면 한번 더 누르면 돌아감
from pycaret.classification import *

# PyCaret에서 사용할 데이터셋 로드
exp1 = setup(data=ml_df, target='취업직종대분류')

# '인문·사회과학 연구직' 레이블을 가진 데이터 제거
ml_df = ml_df[ml_df['취업직종대분류'] != '인문·사회과학 연구직']

# 모델 생성
rf = create_model('rf', fold=5)

,Description,Value
0,Session id,7800
1,Target,취업직종대분류
2,Target type,Multiclass
3,Target mapping,"건설·채굴 연구개발직 및 공학기술직: 0, 건설·채굴직: 1, 경영·행정·사무직: 2, 경호·경비직: 3, 관리직(임원·부서장): 4, 교육직: 5, 금속·재료 설치·정비·생산직(판금·단조·주조·용접·도장 등): 6, 금융·보험직: 7, 기계 설치·정비·생산직: 8, 농림어업직: 9, 돌봄 서비스직(간병·육아): 10, 미용·예식 서비스직: 11, 법률직: 12, 보건·의료직: 13, 사회복지·종교직: 14, 섬유·의복 생산직: 15, 스포츠·레크리에이션직: 16, 식품 가공·생산직: 17, 여행·숙박·오락 서비스직: 18, 영업·판매직: 19, 예술·디자인·방송직: 20, 운전·운송직: 21, 음식 서비스직: 22, 인쇄·목재·공예 및 기타 설치·정비·생산직: 23, 전기·전자 설치·정비·생산직: 24, 정보통신 설치·정비직: 25, 정보통신 연구개발직 및 공학기술직: 26, 제조 단순직: 27, 제조 연구개발직 및 공학기술직: 28, 청소 및 기타 개인서비스직: 29, 화학·환경 설치·정비·생산직: 30"
4,Original data shape,"(45113, 5)"
5,Transformed data shape,"(45113, 39)"
6,Transformed train set shape,"(31579, 39)"
7,Transformed test set shape,"(13534, 39)"
8,Ordinal features,1
9,Numeric features,1


In [12]:
# 모델 3개 acc 높은거 찾기
top3 = compare_models(sort='Accuracy', n_select=3, fold=5)

TypeError: The behavior of tune_model in version 1.0.1 is changed. Please pass trained model object.

In [ ]:
# top3 하이퍼파라미터 튜닝, totalling 100 fits
tuned_top3 = [tune_model(i, n_iter=5) for i in top3]

In [14]:
# 교차검증
tuned_rf = tune_model(rf, fold=5, optimize= 'Accuracy')

NameError: name 'rf' is not defined

In [ ]:
#모델 Blening
#선택한 모델을 조합하여 더욱 강력한 앙상블 모델 만들기, 기본적으로 10번 학습함
blender_top3 = blend_models(estimator_list=top3, verbose=True, method='auto', fold=10)

In [ ]:
#모델 다시 tuning_예측성능을 올리기
blender_tune3 = tune_model(blender_top3)

**최종모델: blender_tune3**

## 최종 모델 예측 및 평가

In [ ]:
#최종 모델 예측 => Voting Classifier
final_model = finalize_model(blender_tune3)
prediction = predict_model(final_model, data=ml_df)

In [ ]:
#타겟값 잘 나오나 확인
prediction = prediction['취업직종대분류']
prediction

In [ ]:
!pip install pycaret==2.3.3

## 모델 저장

In [ ]:
# 지금 만든 모델을 저장
final_model

In [ ]:
from pycaret.classification import save_model

# 모델 저장
save_model(final_model, '/content/drive/MyDrive/Pycret/saved_model')

In [ ]:
from pycaret.classification import load_model

# 모델 불러오기 -> 잘 불려지나 확인
loaded_model = load_model('/content/drive/MyDrive/Pycret/saved_model')
loaded_model

##[나중에]내 이력 맞춤 추천 1가지

In [ ]:
# from pycaret.classification import predict_model
# user_input = {
#     '연령': float(input("연령을 입력하세요: ")),
#     '장애유형': input("장애유형을 입력하세요: "),
#     '중증여부': input("중증여부를 입력하세요: "),
#     '근무지역': input("근무지역을 입력하세요: "),
# }

# user_data = pd.DataFrame([user_input])

In [ ]:
# # 예측 수행
# predictions = predict_model(final_model, data=user_data)

# # 예측 결과 출력
# predictions